In [18]:
import geopandas as gpd
import pandas as pd
import osmnx as ox

In [19]:
# city = 'borsele'
# city = 'breda'
city = 'alphen-waddinxveen'

In [20]:
# boundaries = ['Borsele, Zeeland, Netherlands']
# boundaries = ['Breda, North Brabant, Netherlands']
boundaries = [
    'Alphen aan den Rijn, Zuid-Holland, Netherlands',
    'Waddinxveen, Zuid-Holland, Netherlands',
    'Boskoop, Zuid-Holland, Netherlands'
] 

In [21]:
osm_data = gpd.read_file(f"/Users/cmartens/Documents/thesis_cf_martens/2.risk_analysis/output/{city}/osm_data_with_risk_{city}.geojson")

In [22]:
no_fly_zones = gpd.read_file("/Users/cmartens/Documents/thesis_cf_martens/3.no_fly_zones/input/gdf_no_fly_zones_nl_27feb.geojson")

In [23]:
no_fly_zones['air_type'].unique()

array(['Firefighting Training Areas', 'General Training Area (EHD)',
       'Air Ambulance Landing Sites', 'High-Risk Areas',
       'Restricted Zones', 'Prohibited Airspace (EHP)',
       'Permanently Reserved Airspace (EHR)', 'Protected Nature Reserves'],
      dtype=object)

In [24]:
counts = no_fly_zones.groupby(no_fly_zones.geometry.type)['area_type'].value_counts()

print("Aantal area_types per geometry type voor risk='no_fly_zone':")
print(counts)

Aantal area_types per geometry type voor risk='no_fly_zone':
              area_type  
MultiPolygon  no_fly_zone    208
Polygon       no_fly_zone    676
Name: count, dtype: int64


In [25]:
boundary = ox.geocode_to_gdf(boundaries)

# 2. Zorg dat beide in hetzelfde CRS staan
no_fly_zones = no_fly_zones.to_crs(boundary.crs)

# 3. Filter no-fly zones die binnen Breda vallen
# (Gebruik 'intersects' als je ook gedeeltelijke overlap wilt toestaan)
no_fly_in_boundarie = no_fly_zones[no_fly_zones.geometry.intersects(boundary.geometry.iloc[0])].copy()

In [26]:
no_fly_in_boundarie['risk'] = 'no_fly_zone'

In [27]:
concatenated_gdf = pd.concat([osm_data, no_fly_in_boundarie], ignore_index=True)

In [28]:
# # 1. Projecteer eerst naar meters (bijv. RD New - EPSG:28992)
# concatenated_gdf = concatenated_gdf.to_crs(epsg=28992)

# # 2. Buffer points (cirkels van 50 meter)
# mask_points = (concatenated_gdf['risk'] == 'no_fly_zone') & (concatenated_gdf.geometry.type == 'Point')

# concatenated_gdf.loc[mask_points, 'geometry'] = concatenated_gdf.loc[mask_points, 'geometry'].apply(lambda pt: pt.buffer(50))

# # 3. Buffer linestrings (smalle polygonen van 5 meter breed)
# mask_lines = (concatenated_gdf['risk'] == 'no_fly_zone') & (concatenated_gdf.geometry.type == 'LineString')
# concatenated_gdf.loc[mask_lines, 'geometry'] = concatenated_gdf.loc[mask_lines, 'geometry'].apply(lambda ln: ln.buffer(2.5))

# # (Optioneel) Terugprojecteren naar WGS 84 (graden) als je export naar GeoJSON wilt
# concatenated_gdf = concatenated_gdf.to_crs(epsg=4326)


In [29]:
concatenated_gdf[concatenated_gdf['risk'] == 'no_fly_zone']

,name,id,description,area_type,category,risk,Height,geometry,air_type
32346,Industriegebied met risicos op zware ongevallen,NaN,OPEN cat. vluchten zijn hier verboden SPEC cat...,no_fly_zone,Industry,no_fly_zone,NaN,"POLYGON ((4.65036 52.14199, 4.65036 52.14194, ...",High-Risk Areas
32347,Gebied met verbod vanuit beveiligingsoverwegingen,NaN,OPEN cat. vluchten zijn hier verboden SPEC cat...,no_fly_zone,Security,no_fly_zone,NaN,"POLYGON ((4.64119 52.13227, 4.64119 52.13221, ...",Restricted Zones
32348,Gebied met verbod vanuit beveiligingsoverwegingen,NaN,OPEN cat. vluchten zijn hier verboden SPEC cat...,no_fly_zone,Security,no_fly_zone,NaN,"POLYGON ((4.63869 52.13421, 4.63869 52.13416, ...",Restricted Zones
32349,geografische zone van een beveiligd gebied of ...,NaN,OPEN cat. vluchten zijn hier verboden SPEC cat...,no_fly_zone,Security,no_fly_zone,NaN,"POLYGON ((4.65608 52.14029, 4.65608 52.14027, ...",Restricted Zones
32350,geografische zone van een beveiligd gebied of ...,NaN,OPEN cat. vluchten zijn hier verboden SPEC cat...,no_fly_zone,Security,no_fly_zone,NaN,"POLYGON ((4.58747 52.12724, 4.58747 52.12722, ...",Restricted Zones
32351,Natura 2000 Area De Wilck,NaN,Natura 2000-besluit 7 mei 2013,no_fly_zone,Environment,no_fly_zone,NaN,"MULTIPOLYGON (((4.54282 52.11817, 4.5426 52.11...",Protected Nature Reserves
32352,Natura 2000 Area Nieuwkoopse Plassen & De Haeck,NaN,Natura 2000-besluit 06 januari 2014,no_fly_zone,Environment,no_fly_zone,NaN,"MULTIPOLYGON (((4.83055 52.16943, 4.8305 52.16...",Protected Nature Reserves


In [30]:
mask = concatenated_gdf['risk'] == 'no_fly_zone'
subset = concatenated_gdf[mask]

counts = subset.groupby(subset.geometry.type)['area_type'].value_counts()

print("Aantal area_types per geometry type voor risk='no_fly_zone':")
print(counts)

Aantal area_types per geometry type voor risk='no_fly_zone':
              area_type  
MultiPolygon  no_fly_zone    2
Polygon       no_fly_zone    5
Name: count, dtype: int64


In [31]:
concatenated_gdf[concatenated_gdf['area_type'] == 'Pedestrian and cycling paths']

,name,id,description,area_type,category,risk,Height,geometry,air_type
0,Herenhof,3.767643e+06,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"POLYGON ((4.66026 52.15059, 4.66037 52.15101, ...",NaN
1,Baronie,3.834145e+06,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"POLYGON ((4.64939 52.13405, 4.64942 52.13447, ...",NaN
2,Burgemeester Colijnstraat,4.224910e+06,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"POLYGON ((4.65927 52.07514, 4.65924 52.07516, ...",NaN
3,None,1.022127e+07,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"POLYGON ((4.66766 52.13058, 4.66765 52.13063, ...",NaN
4,None,1.072745e+07,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"POLYGON ((4.64772 52.02732, 4.64776 52.02732, ...",NaN
...,...,...,...,...,...,...,...,...,...
22562,Weteringpad,1.391044e+09,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"LINESTRING (4.63129 52.12083, 4.63154 52.12095)",NaN
22568,None,1.391651e+09,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"LINESTRING (4.65387 52.1124, 4.65398 52.11234,...",NaN
22569,None,1.391806e+09,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"LINESTRING (4.65727 52.12529, 4.65713 52.12529...",NaN
22570,None,1.391806e+09,None,Pedestrian and cycling paths,Transportation,0.463,30.0,"LINESTRING (4.65755 52.12509, 4.65751 52.12505...",NaN


In [32]:
concatenated_gdf.to_file(f"/Users/cmartens/Documents/thesis_cf_martens/3.no_fly_zones/output/data_for_graph_{city}.geojson", driver="GeoJSON")